<a href="https://colab.research.google.com/github/sandeepchemistry/CVPAT/blob/master/gpu4pyscf_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!nvcc --version

Wed Nov 22 17:40:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install gpu4pyscf-cuda11x

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.1/429.1 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 MB 8.7 MB/s eta 0:00:00
  Created wheel for geometric: filename=geometric-1.0.1-py3-none-any.whl size=445147 sha256=49986b439d0a7bbbf86e855e394dc81f2421d4c0b256322331e19009bc443040
  Stored in directory: /root/.cache/pip/wheels/1d/e8/a2/f21341a562a7245943dfbf758934653695345e177a0989a509
Successfully built geometric
  Attempting uninstall: cupy-cuda11x
    Found existing installation: cupy-cuda11x 11.0.0
    Uninsta

In [ ]:
# https://docs.cupy.dev/en/stable/install.html#installing-cupy
!python -m cupyx.tools.install_library --cuda 11.x --library cutensor

By downloading and using cuTENSOR, you accept the terms and conditions of the NVIDIA cuTENSOR Software License Agreement:
  https://docs.nvidia.com/cuda/cutensor/license.html

Installing cutensor 1.6.2 for CUDA 11.x to: /root/.cupy/cuda_lib/11.x/cutensor/1.6.2
Extracting...
Installing...
Cleaning up...
Done!


PLEASE manually restart the kernel to update the environment variables ONLY if there is still error.

selecting "Runtime" from the menu, then choosing "Restart runtime"

In [ ]:
# Changed from gpu4pyscf/example/00-h2o.py
# Copyright 2023 The GPU4PySCF Authors. All Rights Reserved.
import pyscf
from pyscf import lib
from gpu4pyscf.dft import rks
lib.num_threads(2)

/usr/local/lib/python3.10/dist-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


2

In [ ]:
atom ='''
O       0.0000000000    -0.0000000000     0.1174000000
H      -0.7570000000    -0.0000000000    -0.4696000000
H       0.7570000000     0.0000000000    -0.4696000000
'''

xc='B3LYP'
bas='def2-tzvpp'
auxbasis='def2-tzvpp-jkfit'
scf_tol = 1e-10
max_scf_cycles = 50
screen_tol = 1e-14
grids_level = 3

mol = pyscf.M(atom=atom, basis=bas, max_memory=32000, output='./pyscf.log')

mol.verbose = 4
mf_GPU = rks.RKS(mol, xc=xc).density_fit(auxbasis=auxbasis)
mf_GPU.grids.level = grids_level
mf_GPU.conv_tol = scf_tol
mf_GPU.max_cycle = max_scf_cycles
mf_GPU.screen_tol = screen_tol

# Compute Energy
e_dft = mf_GPU.kernel()
print(f"total energy = {e_dft}")

# Compute Gradient
g = mf_GPU.nuc_grad_method()
g.max_memory = 20000
g.auxbasis_response = True
g_dft = g.kernel()

# Compute Hessian
h = mf_GPU.Hessian()
h.auxbasis_response = 2
h_dft = h.kernel()

output file: ./pyscf.log


Overwritten attributes  screen_tol  of <class 'gpu4pyscf.df.df_jk.DFRKS'>
<class 'gpu4pyscf.df.df_jk.DFRKS'> does not have attributes  disp rhoj rhok


total energy = -76.4666820229345
